### SAS 3


- PERMNO 누락 건 제거 
- BE 집계 후, 0 미만 또는 nan 값인 경우 삭제  
- 하나의 year(datadate) 에 여러개의 datadate이 존재할 수 있으므로 최신 것만 유지하고 삭제 (permno, gvkey, year 그룹) 

In [1]:
import pandas as pd
import numpy as np 

In [2]:
comp = pd.read_csv(r"C:\Users\User\OneDrive\Desktop/compustat_permno.csv")
sample_data = pd.read_csv(r"C:\Users\User\OneDrive\Desktop/assignment1_sample_data.csv")
crsp = pd.read_csv(r"C:\Users\User\OneDrive\Desktop/CRSP_M.csv")

In [3]:
print(comp.dtypes)
print(comp.shape)
comp.head()

gvkey         int64
datadate      int64
itcb        float64
pstk        float64
pstkl       float64
pstkrv      float64
seq         float64
txdb        float64
permno      float64
permco      float64
dtype: object
(434269, 10)


,gvkey,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb,permno,permco
0,1000,19611231,0.0,NaN,0.0,NaN,NaN,0.000,NaN,NaN
1,1000,19621231,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,1000,19631231,0.0,0.0,0.0,0.0,0.553,0.008,NaN,NaN
3,1000,19641231,0.0,0.0,0.0,0.0,0.607,0.020,NaN,NaN
4,1000,19651231,0.0,0.0,0.0,0.0,0.491,0.000,NaN,NaN


In [4]:
comp = comp.dropna(subset=['permno']) # drop na permno
print(comp['permno'].isna().sum())
print(comp.shape)

0
(264450, 10)


In [5]:
comp['datadate'] = pd.to_datetime(comp['datadate'], format='%Y%m%d')
comp['year'] = comp['datadate'].dt.year
print(comp.shape)

(264450, 11)


In [6]:
col = ['pstkrv', 'pstkl', 'pstk', 'seq', 'txdb', 'itcb']
comp['pref'] = comp.get("pstkrv", np.nan)
if 'pstkl' in comp.columns:
    comp['pref'] = comp['pref'].fillna(comp['pstkl'])
if 'pstk' in comp.columns:
    comp['pref'] = comp['pref'].fillna(comp['pstk'])


print(comp.shape)

(264450, 12)


In [7]:
# compute BE = SEQ + TXDB + ITCB - Preferred Stock
comp['BE'] = comp['seq'] + comp['txdb']+ comp['itcb'].fillna(0) - comp['pref'].fillna(0)
query = comp['BE'] > 0 
comp = comp.loc[query, ]
print(comp.shape)

(213734, 13)


In [8]:
query = comp['BE'].isna() == True
comp.loc[query, ]

,gvkey,datadate,itcb,pstk,pstkl,pstkrv,seq,txdb,permno,permco,year,pref,BE


In [9]:
 # keep the last observation in each group 
idx = comp.groupby(['gvkey', 'permno', 'year'])['datadate'].idxmax()
comp = comp.loc[idx].reset_index(drop=True)
print(comp.shape)

(213271, 13)


### SAS 5 

- 월별 수익 데이터(CRSP)를 기준으로 ME 계산
- 각 월말 (date) 별로 하나의 PERMNO에 대해 ME를 계산한 후 
- date , permco 를 기준으로 ME를 sum
- 이후에 해당 date, permco 기준 me가 가장 큰 PERMNO만 남기고 삭제 , me_sum을 할당 

In [10]:
print(crsp.dtypes)
crsp.head()

DATE        int64
DLSTCD    float64
PERMNO      int64
SHRCD       int64
EXCHCD      int64
SICCD     float64
DLRET     float64
PERMCO      int64
PRC       float64
VOL       float64
RET       float64
SHROUT    float64
ALTPRC    float64
rf        float64
dtype: object


,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf
0,19610131,NaN,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.25,0.0019
1,19610131,NaN,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.00,0.0019
2,19610131,NaN,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.75,0.0019
3,19610131,NaN,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.00,0.0019
4,19610131,NaN,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.50,0.0019


In [11]:
crsp['DATE'] = pd.to_datetime(crsp['DATE'], format='%Y%m%d')

In [12]:
col = ['PRC', 'SHROUT', 'ALTPRC', 'ME']
query = (crsp['ALTPRC'].abs() > 0) & (crsp['SHROUT']> 0)
crsp.loc[query, 'ME'] = crsp['ALTPRC'].abs() * crsp['SHROUT'] / 1000 # calculate ME in million dollars
crsp.head()

# calculating return considering delisiting 
crsp['DLSTCD'] = crsp['DLSTCD'].astype('Int64')  
query = crsp['DLRET'].isna() & crsp['DLSTCD'].notna()
query_30 = (
    crsp['DLSTCD'].isin([500, 520, 574, 580, 584]) |
    crsp['DLSTCD'].between(551, 573)
)
crsp.loc[query & query_30, 'DLRET'] = -0.3
crsp.loc[query & ~query_30, 'DLRET'] = -1 

In [13]:
#compute adjusted return considering delisting return
crsp['adj_ret'] = crsp['RET']
query_both = crsp['DLSTCD'].notna() & crsp['DLRET'].notna()& crsp['RET'].notna()
crsp.loc[query_both, 'adj_ret'] = (1 + crsp['RET']) * (1 + crsp['DLRET']) - 1

query_only = crsp['DLSTCD'].notna() & crsp['DLRET'].notna()& crsp['RET'].isna()
crsp.loc[query_only, 'adj_ret'] = crsp['DLRET'] 

In [14]:
crsp['excess_ret'] = crsp['adj_ret'] - crsp['rf']

In [15]:
print(crsp.shape)
crsp.head()

(2921193, 17)


,DATE,DLSTCD,PERMNO,SHRCD,EXCHCD,SICCD,DLRET,PERMCO,PRC,VOL,RET,SHROUT,ALTPRC,rf,ME,adj_ret,excess_ret
0,1961-01-31,<NA>,10006,10,1,3740.0,NaN,22156,50.25,939.0,0.322368,1420.0,50.25,0.0019,71.35500,0.322368,0.320468
1,1961-01-31,<NA>,10014,10,1,3710.0,NaN,22157,4.00,395.0,0.000000,2504.0,4.00,0.0019,10.01600,0.000000,-0.001900
2,1961-01-31,<NA>,10030,10,1,3310.0,NaN,22160,41.75,280.0,0.087948,1627.0,41.75,0.0019,67.92725,0.087948,0.086048
3,1961-01-31,<NA>,10057,11,1,3540.0,NaN,20020,54.00,152.0,0.142857,500.0,54.00,0.0019,27.00000,0.142857,0.140957
4,1961-01-31,<NA>,10102,10,1,2810.0,NaN,22164,79.50,480.0,0.032468,3965.0,79.50,0.0019,315.21750,0.032468,0.030568


In [16]:
# aggregate ME for a given permco and date
crsp = crsp.dropna(subset=['ME'])
me_agg =crsp.groupby(['PERMCO', 'DATE'])['ME'].sum().reset_index().rename(columns={'ME': 'ME_sum'})

In [17]:
me_agg.head()

,PERMCO,DATE,ME_sum
0,3,1977-01-31,6.412500
1,3,1977-02-28,6.946875
2,3,1977-03-31,6.056250
3,3,1977-04-29,6.501563
4,3,1977-05-31,6.857812


In [18]:
query = crsp.groupby(['PERMCO', 'DATE'])['ME'].idxmax() #sort ME within PERMCO and DATE, get the index of the max ME
crsp_max = crsp.loc[query, ['PERMNO', 'DATE', 'PERMCO']]
crsp_max.head()

,PERMNO,DATE,PERMCO
534930,12160,1977-01-31,3
539767,12160,1977-02-28,3
544586,12160,1977-03-31,3
549404,12160,1977-04-29,3
554226,12160,1977-05-31,3


In [19]:
crsp_agg = crsp_max.merge(me_agg, on =['PERMCO', 'DATE'], how='inner') #total market equity for each permco and date
print(crsp_agg.shape)
crsp_agg.head()

(2878883, 4)


,PERMNO,DATE,PERMCO,ME_sum
0,12160,1977-01-31,3,6.412500
1,12160,1977-02-28,3,6.946875
2,12160,1977-03-31,3,6.056250
3,12160,1977-04-29,3,6.501563
4,12160,1977-05-31,3,6.857812


In [20]:
crsp = crsp_agg.merge(crsp.drop(columns = ['ME']), on= ['PERMNO', 'DATE', 'PERMCO'], how = 'inner').rename(columns={'ME_sum': 'ME'})
crsp.shape

(2878883, 17)

In [21]:
crsp.head()

,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,VOL,RET,SHROUT,ALTPRC,rf,adj_ret,excess_ret
0,12160,1977-01-31,3,6.412500,<NA>,10,3,NaN,NaN,-4.5000,NaN,NaN,1425.0,-4.5000,0.0036,NaN,NaN
1,12160,1977-02-28,3,6.946875,<NA>,10,3,NaN,NaN,-4.8750,NaN,0.083333,1425.0,-4.8750,0.0035,0.083333,0.079833
2,12160,1977-03-31,3,6.056250,<NA>,10,3,NaN,NaN,-4.2500,NaN,-0.128205,1425.0,-4.2500,0.0038,-0.128205,-0.132005
3,12160,1977-04-29,3,6.501563,<NA>,10,3,NaN,NaN,-4.5625,NaN,0.073529,1425.0,-4.5625,0.0038,0.073529,0.069729
4,12160,1977-05-31,3,6.857812,<NA>,10,3,NaN,NaN,-4.8125,NaN,0.054795,1425.0,-4.8125,0.0037,0.054795,0.051095


In [22]:

#validation 
val = crsp.groupby(['PERMNO', 'DATE'])['ME'].nunique().reset_index(name='n_me')
val.sort_values('n_me', ascending=False)

,PERMNO,DATE,n_me
0,10000,1986-01-31,1
1,10000,1986-02-28,1
2,10000,1986-03-31,1
3,10000,1986-04-30,1
4,10000,1986-05-30,1
...,...,...,...
2878878,93436,2012-08-31,1
2878879,93436,2012-09-28,1
2878880,93436,2012-10-31,1
2878881,93436,2012-11-30,1


### SAS 6

- 이전에 계산한 me를 기준으로 
    - 해당 년도 6월의 ME를 추출 
    - 이전 년도 12월의 ME를 추출 , year+1 으로 하여 연결 
    - comp 테이블의 BE는 year + 1 으로 하여 연결 
- 해당 결과값을 가지고 , crsp와 연결할 때 
    - 6월 이후 시점인 경우 당년도 t와 매칭 
    - 6월 이전 시점인 경우 전년도 t와 매칭 
- 이후 마지막으로 수정 주가는 lag(1), ME 또한 동일하게 lag(1)


In [23]:
# adding date related columns
crsp['year'] = crsp['DATE'].dt.year
crsp['month'] = crsp['DATE'].dt.month

In [24]:
# ME_june 
query = (crsp['month'] == 6) &  (crsp['ME'].notna())
me_june = crsp.loc[query, ['PERMNO', 'DATE', 'ME']].rename(columns={'ME': 'ME_june'})
me_june['t'] = me_june['DATE'].dt.year
me_june.drop(columns='DATE', inplace=True)
print(me_june.shape)
me_june = me_june.drop_duplicates(['PERMNO', 't']) 
print(me_june.shape)


(239521, 3)
(239521, 3)


In [25]:
# ME_last_dec 
me_dec = crsp.loc[(crsp['month'] == 12) & (crsp['ME'].notna()), ['PERMNO', 'DATE', 'ME']].rename(columns={'ME': 'ME_last_dec'})
me_dec['t'] = me_dec['DATE'].dt.year + 1 
me_dec.drop(columns='DATE', inplace=True)
print(me_dec.shape)
me_dec = me_dec.drop_duplicates(['PERMNO', 't'])  
print(me_dec.shape)

(242805, 3)
(242805, 3)


In [26]:
# BE_last_year 

be_prev = comp[['permno', 'year', 'BE']].rename(columns={'permno': 'PERMNO', 'BE': 'BE_last_year'})
be_prev['t'] = be_prev['year'] + 1 
be_prev.drop(columns='year', inplace=True)
print(be_prev.shape)
be_prev = be_prev.drop_duplicates(['PERMNO', 't']) 
print(be_prev.shape)

(213271, 3)
(213264, 3)


In [27]:
# merge all together 
me_bm = (me_june.merge(be_prev, on=['PERMNO','t'], how='inner', validate='one_to_one')
                .merge(me_dec , on=['PERMNO','t'], how='inner', validate='one_to_one'))
# me_bm = me_june.merge(be_prev, on=['PERMNO', 't'], how='inner').merge(me_dec, on=['PERMNO', 't'], how='inner')
me_bm['BM'] = me_bm['BE_last_year'] / me_bm['ME_last_dec'] #ME_last_dec 로 수정 필요 
me_bm = me_bm[['PERMNO', 't', 'ME_june', 'BM']]
me_bm.head()

,PERMNO,t,ME_june,BM
0,12160,1978,8.371875,2.640180
1,12160,1979,11.400000,2.241985
2,12160,1980,15.682500,1.217452
3,12160,1982,23.364000,1.233353
4,63845,1973,77.961625,0.135024


In [28]:
# validation 

val = me_bm.groupby(['PERMNO', 't'])['ME_june'].nunique().reset_index(name = 'n_me')
val.sort_values('n_me', ascending=False)

,PERMNO,t,n_me
0,10000,1987,1
1,10001,1987,1
2,10001,1988,1
3,10001,1989,1
4,10001,1990,1
...,...,...,...
174194,93434,2011,1
174195,93434,2012,1
174196,93435,2011,1
174197,93436,2011,1


In [29]:
crsp['t'] = np.where(crsp['month'] > 6, crsp['year'], crsp['year'] -1)
crsp.head()

,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,VOL,RET,SHROUT,ALTPRC,rf,adj_ret,excess_ret,year,month,t
0,12160,1977-01-31,3,6.412500,<NA>,10,3,NaN,NaN,-4.5000,NaN,NaN,1425.0,-4.5000,0.0036,NaN,NaN,1977,1,1976
1,12160,1977-02-28,3,6.946875,<NA>,10,3,NaN,NaN,-4.8750,NaN,0.083333,1425.0,-4.8750,0.0035,0.083333,0.079833,1977,2,1976
2,12160,1977-03-31,3,6.056250,<NA>,10,3,NaN,NaN,-4.2500,NaN,-0.128205,1425.0,-4.2500,0.0038,-0.128205,-0.132005,1977,3,1976
3,12160,1977-04-29,3,6.501563,<NA>,10,3,NaN,NaN,-4.5625,NaN,0.073529,1425.0,-4.5625,0.0038,0.073529,0.069729,1977,4,1976
4,12160,1977-05-31,3,6.857812,<NA>,10,3,NaN,NaN,-4.8125,NaN,0.054795,1425.0,-4.8125,0.0037,0.054795,0.051095,1977,5,1976


In [30]:
ret_me_bm = crsp.merge(me_bm, on=['PERMNO', 't'], how='inner') #219616
print(ret_me_bm.shape)

(1985625, 22)


In [31]:
# validation 
val = ret_me_bm.copy()
val['period'] = np.where(ret_me_bm['month'] <= 6 , 'pre_june', 'post_june')

val.groupby(['PERMNO', 'year', 'period'])['t'].nunique().unstack()
query = (val['PERMNO'] == 10001) & (val['year'] == 1988)
val.loc[query,].head(10)

,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,...,ALTPRC,rf,adj_ret,excess_ret,year,month,t,ME_june,BM,period
645313,10001,1988-01-29,7953,6.20000,<NA>,11,3,4920.0,NaN,6.2500,...,6.2500,0.0029,0.063830,0.060930,1988,1,1987,5.822125,1.014415,pre_june
645314,10001,1988-02-29,7953,6.69600,<NA>,11,3,4920.0,NaN,6.7500,...,6.7500,0.0046,0.080000,0.075400,1988,2,1987,5.822125,1.014415,pre_june
645315,10001,1988-03-31,7953,6.07600,<NA>,11,3,4920.0,NaN,6.1250,...,6.1250,0.0044,-0.076296,-0.080696,1988,3,1987,5.822125,1.014415,pre_june
645316,10001,1988-04-29,7953,6.26200,<NA>,11,3,4920.0,NaN,-6.3125,...,-6.3125,0.0046,0.030612,0.026012,1988,4,1987,5.822125,1.014415,pre_june
645317,10001,1988-05-31,7953,6.38600,<NA>,11,3,4920.0,NaN,-6.4375,...,-6.4375,0.0051,0.019802,0.014702,1988,5,1987,5.822125,1.014415,pre_june
645318,10001,1988-06-30,7953,6.20000,<NA>,11,3,4920.0,NaN,6.2500,...,6.2500,0.0049,-0.012039,-0.016939,1988,6,1987,5.822125,1.014415,pre_june
645319,10001,1988-07-29,7953,6.38600,<NA>,11,3,4920.0,NaN,-6.4375,...,-6.4375,0.0051,0.030000,0.024900,1988,7,1988,6.200000,1.207618,post_june
645320,10001,1988-08-31,7953,6.57200,<NA>,11,3,4920.0,NaN,6.6250,...,6.6250,0.0059,0.029126,0.023226,1988,8,1988,6.200000,1.207618,post_june
645321,10001,1988-09-30,7953,6.36225,<NA>,11,3,4920.0,NaN,-6.3750,...,-6.3750,0.0062,-0.021132,-0.027332,1988,9,1988,6.200000,1.207618,post_june
645322,10001,1988-10-31,7953,6.61175,<NA>,11,3,4920.0,NaN,6.6250,...,6.6250,0.0061,0.039216,0.033116,1988,10,1988,6.200000,1.207618,post_june


In [32]:
ret_me_bm = ret_me_bm.sort_values(['PERMNO','DATE']).drop_duplicates(['PERMNO','DATE'])
print(ret_me_bm.shape)
ret_me_bm.head()

(1985625, 22)


,PERMNO,DATE,PERMCO,ME,DLSTCD,SHRCD,EXCHCD,SICCD,DLRET,PRC,...,SHROUT,ALTPRC,rf,adj_ret,excess_ret,year,month,t,ME_june,BM
645307,10001,1987-07-31,7953,5.9460,<NA>,11,3,4920.0,NaN,6.0000,...,991.0,6.0000,0.0046,0.021277,0.016677,1987,7,1987,5.822125,1.014415
645308,10001,1987-08-31,7953,6.4415,<NA>,11,3,4920.0,NaN,6.5000,...,991.0,6.5000,0.0047,0.083333,0.078633,1987,8,1987,5.822125,1.014415
645309,10001,1987-09-30,7953,6.2000,<NA>,11,3,4920.0,NaN,6.2500,...,992.0,6.2500,0.0045,-0.022308,-0.026808,1987,9,1987,5.822125,1.014415
645310,10001,1987-10-30,7953,6.3240,<NA>,11,3,4920.0,NaN,6.3750,...,992.0,6.3750,0.0060,0.020000,0.014000,1987,10,1987,5.822125,1.014415
645311,10001,1987-11-30,7953,6.1380,<NA>,11,3,4920.0,NaN,6.1875,...,992.0,6.1875,0.0035,-0.029412,-0.032912,1987,11,1987,5.822125,1.014415


In [33]:
ret_me_bm['altprc_lag1'] = ret_me_bm.groupby('PERMNO')['ALTPRC'].shift(1)
ret_me_bm['ME_lag1'] = ret_me_bm.groupby('PERMNO')['ME'].shift(1)
ret_me_bm['prev_date'] = ret_me_bm.groupby('PERMNO')['DATE'].shift(1)

In [34]:
ret_me_bm['gap_month'] = ((ret_me_bm['year'] - ret_me_bm['prev_date'].dt.year)*12 
+ (ret_me_bm['month'] - ret_me_bm['prev_date'].dt.month))
ret_me_bm.head()
query = (ret_me_bm['gap_month'] > 1) | (ret_me_bm['gap_month'].isna() == True)
ret_me_bm.loc[query, ['altprc_lag1', 'ME_lag1']] = np.nan

In [35]:
#validation 
ret_me_bm.groupby(['PERMNO', 'DATE']).size().reset_index(name='n_index').sort_values(by = 'n_index', ascending=False)
ret_me_bm=ret_me_bm.drop_duplicates(['PERMNO', 'DATE']) 

In [36]:
query = ret_me_bm[['adj_ret', 'ME_lag1', 'ME_june', 'BM']].notna().all(axis=1)
assignment_data = ret_me_bm.loc[query, ['PERMNO', 'DATE', 'year', 'EXCHCD', 'SICCD', 'adj_ret', 'excess_ret', 'altprc_lag1', 'ME_lag1', 'ME_june', 'BM' ]].sort_values(by=['PERMNO', 'DATE']).reset_index(drop=True)

print(assignment_data.shape)
assignment_data.head()

assignment_agg = assignment_data.copy()

(1963862, 11)


In [37]:
log_cnt = 1983365 #from sas code 
log_data_diff = (assignment_data.shape[0] - log_cnt)/ assignment_data.shape[0]
print(f"log_difference : {log_data_diff:.4f}")

log_difference : -0.0099


In [38]:
assignment_data.drop_duplicates(['PERMNO','DATE']).shape
assignment_data.to_csv("assignment1_data.csv", index=False)

In [39]:
assignment_data.head(25).to_csv("first_25_obs.csv", index =False)

In [40]:
assignment_agg['yyyymm'] = assignment_agg['DATE'].dt.strftime('%Y%m')
selected_year = ['197012', '198012','199012','200012','201012']
query = assignment_agg['yyyymm'].isin(selected_year)

In [41]:
assignment_agg.loc[query, ].groupby('yyyymm').agg(
    mean = ('excess_ret', 'mean'),
    std = ('excess_ret', 'std'),
    min = ('excess_ret', 'min'),
    max = ('excess_ret', 'max'), 
    N_permnos = ('PERMNO', 'nunique')
).reset_index()

assignment_agg.loc[query, ].groupby('yyyymm').agg(
    mean = ('excess_ret', 'mean'),
    std = ('excess_ret', 'std'),
    min = ('excess_ret', 'min'),
    max = ('excess_ret', 'max'), 
    N_permnos = ('PERMNO', 'nunique')
).reset_index().to_csv("summary_stats.csv", index=False) 

### validation w/ sample data

In [42]:
sample_data.describe()

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM
count,3198.000000,3198.0,3198.0,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000,3198.000000
mean,68969.367417,20121231.0,2012.0,2.227955,5035.565041,0.026761,0.026661,66.307163,4348.820794,4217.002132,0.923025
std,27968.735150,0.0,0.0,0.940657,2181.075090,0.117142,0.117142,2332.963722,20326.911889,20087.698277,0.924562
min,10001.000000,20121231.0,2012.0,1.000000,170.000000,-0.774834,-0.774934,-23.654999,0.888375,0.854465,0.000795
25%,53503.750000,20121231.0,2012.0,1.000000,3545.000000,-0.015974,-0.016074,5.200000,103.046618,102.681551,0.381303
50%,82242.500000,20121231.0,2012.0,3.000000,4924.000000,0.021051,0.020951,14.410000,464.349117,446.737531,0.705639
75%,89695.750000,20121231.0,2012.0,3.000000,6330.000000,0.065244,0.065144,32.355000,1959.077633,1891.924352,1.132053
max,93436.000000,20121231.0,2012.0,3.000000,9999.000000,2.520000,2.519900,131916.000000,550568.241320,547363.344000,10.565308


In [43]:
sample_data['date'] = pd.to_datetime(sample_data['date'], format='%Y%m%d')
sample_data['yyyymm'] = sample_data['date'].dt.strftime('%Y%m')
sample_data.groupby('yyyymm').agg(
    mean = ('eretadj', 'mean'),
    std = ('eretadj', 'std'),
    min = ('eretadj', 'min'),
    max = ('eretadj', 'max'), 
    N_permnos = ('permno', 'nunique')
)

,mean,std,min,max,N_permnos
yyyymm,,,,,
201212,0.026661,0.117142,-0.774934,2.5199,3198


In [44]:
query = assignment_agg['yyyymm'] == '201212'
assignment_agg.loc[query, ].groupby('yyyymm').agg(
    mean = ('excess_ret', 'mean'),
    std = ('excess_ret', 'std'),
    min = ('excess_ret', 'min'),
    max = ('excess_ret', 'max'), 
    N_permnos = ('PERMNO', 'nunique')
).reset_index() 

,yyyymm,mean,std,min,max,N_permnos
0,201212,0.026661,0.117142,-0.774934,2.5199,3198
